In [ ]:
##data Loading Techniques

In [13]:
#No.1 TextLoader
from langchain_community.document_loaders import TextLoader
loader = TextLoader("speech.txt")
text_doc=loader.load;
text_doc

<bound method BaseLoader.load of <langchain_community.document_loaders.text.TextLoader object at 0x0000013431A300D0>>

In [1]:
#No.2 Web Loder defult directly load all page
from langchain_community.document_loaders import WebBaseLoader
loader=WebBaseLoader("https://www.geeksforgeeks.org/data-science-interview-questions-and-answers/")
data=loader.load()
print(data)

[Document(page_content="\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nTop 100+ Data Science Interview Questions and Answers (2024)\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content\n\n\n\n\n\n\n\n\n\nCoursesDSA to DevelopmentFor Working ProfessionalsData Structure & Algorithm Classes (Live)System Design (Live)JAVA Backend Development(Live)DevOps(Live)Data Structures & Algorithms in PythonFor StudentsInterview Preparation CourseGATE CS & IT 2024Data Science (Live)Data Structure & Algorithm-Self Paced(C++/JAVA)Master Competitive Programming(Live)Full Stack Development with React & Node JS(Live)For School StudentsCBSE Class 12 Computer ScienceSchool GuidePython Programming FoundationAll CoursesTutorialsPython TutorialTaking Input in PythonPython OperatorsPython VariablesPython Data TypesData TypesPython NumbersPython StringPython ListsPython TuplesSets in PythonPython DictionaryPython Loops 

In [12]:
#No.2.1 web loader Specific class wise loader more ifficient
from langchain_community.document_loaders import WebBaseLoader
import bs4
loader=WebBaseLoader(web_path=("https://www.geeksforgeeks.org/data-science-interview-questions-and-answers/",)
                     ,bs_kwargs=dict(parse_only= bs4.SoupStrainer(class_=("article-title","text"))))
data=loader.load()
print(data)

[Document(page_content='\nData Science Interview Questions and Answers\n\nData Science Interview Questions – Explore the Data Science Interview Questions and Answers for beginners and experienced professionals looking for new opportunities in data science. \n\nWe all know that data science is a field where data scientists mine raw data, analyze it, and extract useful insights from it. The article outlines the frequently asked questionas during the data science interview. Practising all the below questions will help you to explore your career as a data scientist.\nTable of Content\nBasic Data Science Interview Questions For FresherIntermediate Data Science Interview Questions Data Science Interview Questions for ExperiencedWhat is Data Science?Data science is a field that extracts knowledge and insights from structured and unstructured data by using scientific methods, algorithms, processes, and systems. It combines expertise from various domains, such as statistics, computer science, m

In [1]:
#No 3 pdf loader
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("reserchpaper.pdf")
pdfload= loader.load()


In [2]:
## text splitting technique

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

textsplitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=100)
splitted=textsplitter.split_documents(pdfload)
print(splitted[:3])

[Document(page_content="Page 1 of 3 \n \n \n \n \n  \nTejas Ahirrao  \n3rd Year Student  \nDept.of COMP  \nSKN Sinhgad Institute of  \nTechnology and Science, Lonavala, Pune. \ntejasahirrao.sknsits@sinhgad.edu  \n \nTejashree Khaire  \n3rd Year Student  \nDept.of COMP  \nSKN Sinhgad Institute of  \nTechnology and Science, Lonavala, Pune.  \ntejashreekhaire.sknsits.comp@gmail.com  \n \nAbstract—In the busy metropolitan cities like Mumbai, Delhi , \npeople don't have time to invest in waiting for transport. Waiting", metadata={'source': 'reserchpaper.pdf', 'page': 0}), Document(page_content="people don't have time to invest in waiting for transport. Waiting \ntime for transport in such crowded cities leads to less productivity \non a whole. People face this problem in their daily life where the y \nhave no idea about the current status of their transport. So the \nproposed solution is an android based application that will help \nthe user to check out the current location of the bus and 

In [4]:
## vector Databse indexing, embbeding and retriver object

In [6]:
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings

vectorstore = Chroma.from_documents(documents=splitted, embedding= OllamaEmbeddings(model="nomic-embed-text",show_progress= True))

retriver = vectorstore.as_retriever() #can use multipul type of serach like MMR or similarity(defult)

OllamaEmbeddings: 100%|██████████| 35/35 [01:14<00:00,  2.13s/it]


In [ ]:
## chain for the rag

In [7]:
from langchain_community.llms import Ollama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
# prompt
prmpt= PromptTemplate.from_template("""You are an assistant for question-answering tasks. Use the following pieces of
                                         retrieved context to answer the question. If you don't know the answer, just say that 
                                         you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} """)

#llm 
llm = Ollama(model="llama3")

# output parser 
outputparser = StrOutputParser()

#chain
rag_chain=(
    {"context": retriver , "question": RunnablePassthrough()}
    |prmpt
    |llm
    |outputparser
)


In [8]:
rag_chain.invoke("what is this document about")

OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.09s/it]


'This document is about a research paper that proposes a "Real Time Bus Tracking System" which allows users to track the location of buses in real-time.'